# Imports 

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
from custom_methods import model_eval

datapath = '../Data/'

startTime = time.time()

# Get Data

In [2]:
filename = 'processed.pickle'
infile = open(datapath+filename,'rb')
df = pickle.load(infile)
infile.close()
df.head()

,SPA_PER_ID,SPA_PREM_ID,MONTH,SPA_ACCT_ID,RES_EL_CUR120_DAYS,RES_EL_CUR22_DAYS,RES_EL_CUR30_DAYS,RES_EL_CUR60_DAYS,RES_EL_CUR90_DAYS,RES_EL_CUR_BAL_AMT,...,TOTAL_30_DAYS_AMT,TOTAL_60_DAYS_AMT,TOTAL_90_DAYS_AMT,TOTAL_CUR_BALANCE,LAST_MO_W_DATA,WITHIN_6_MO_PRIOR_LAST_DATA,MO_AWAY,NUM_PREM_FOR_PER,NUM_PER_FOR_PREM,MULTI_DWELL_SIZE
0,3,98612,0,171648,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,103.40,False,False,NaN,1.0,1.0,52212
1,3,98612,1,171648,0.0,65.26,0.0,0.0,0.0,65.26,...,0.0,0.0,0.0,307.71,False,False,NaN,1.0,1.0,52212
2,3,98612,2,171648,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,105.80,False,False,NaN,1.0,1.0,52212
3,3,98612,3,171648,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,105.82,False,False,NaN,1.0,1.0,52212
4,3,98612,4,171648,0.0,54.89,0.0,0.0,0.0,54.89,...,0.0,0.0,0.0,226.47,False,False,NaN,1.0,1.0,52212


In [3]:
nulls = df.isnull().sum()
nulls[nulls != 0]

MO_AWAY    3222658
dtype: int64

MO_AWAY is the only column with nulls - good

In [4]:
df.columns.to_list()

['SPA_PER_ID',
 'SPA_PREM_ID',
 'MONTH',
 'SPA_ACCT_ID',
 'RES_EL_CUR120_DAYS',
 'RES_EL_CUR22_DAYS',
 'RES_EL_CUR30_DAYS',
 'RES_EL_CUR60_DAYS',
 'RES_EL_CUR90_DAYS',
 'RES_EL_CUR_BAL_AMT',
 'RES_EL_OVER_120_DAYS',
 'RES_GAS_CUR120_DAYS',
 'RES_GAS_CUR22_DAYS',
 'RES_GAS_CUR30_DAYS',
 'RES_GAS_CUR60_DAYS',
 'RES_GAS_CUR90_DAYS',
 'RES_GAS_CUR_BAL_AMT',
 'RES_GAS_OVER_120_DAYS',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'CITY_TOT_DUE',
 'CITY_30_DAYS_PAST_DUE_AMT',
 'CITY_60_DAYS_PAST_DUE_AMT',
 'CITY_90_DAYS_PAST_DUE_AMT',
 'CMIS_MATCH',
 'APARTMENT',
 'ENROLL_DATE',
 'HAS_COTENANT',
 'BLOCKGROUP_GEOID',
 'POSTAL',
 'MULTI_DWELL_ID',
 'AGG_INCOME_GEO',
 'NO_EARNINGS_GEO',
 'BELOW_POVERTY_LVL_GEO',
 'SNAP_GEO',
 'ABOVE_GRD7_GEO',
 'PUBLIC_ASSIST_GEO',
 'AVISTA_CUR120_DAYS',
 'AVISTA_OVER_120_DAYS',
 'AVISTA_CUR22_DAYS',
 'AVISTA_CUR30_DAYS',
 'AVISTA_CUR60_DAYS',
 'AVIST

# Select Subset of Attributes

In [5]:
keep = [
    'MONTH',
    #
    #'RES_EL_CUR30_DAYS',
    #'RES_EL_CUR60_DAYS',
    #'RES_EL_CUR90_DAYS',
    #'RES_EL_CUR_BAL_AMT',
    #'RES_GAS_CUR30_DAYS',
    #'RES_GAS_CUR60_DAYS',
    #'RES_GAS_CUR90_DAYS',
    #'RES_GAS_CUR_BAL_AMT',
    #
    'BREAK_ARRANGEMENT',
    'BREAK_PAY_PLAN',
    'CALL_OUT',
    'CALL_OUT_MANUAL',
    'DUE_DATE',
    'FINAL_NOTICE',
    'PAST_DUE',
    'SEVERANCE_ELECTRIC',
    'SEVERANCE_GAS',
    #
    #'CITY_TOT_DUE',
    #'CITY_30_DAYS_PAST_DUE_AMT',
    #'CITY_60_DAYS_PAST_DUE_AMT',
    #'CITY_90_DAYS_PAST_DUE_AMT',
    #
    'TOTAL_30_DAYS_AMT',
    'TOTAL_60_DAYS_AMT',
    'TOTAL_90_DAYS_AMT',
    'TOTAL_CUR_BALANCE',
    #
    'SPA_PER_ID',
    'SPA_PREM_ID',
    'CMIS_MATCH',
    #
    'HAS_COTENANT',
    'NUM_PREM_FOR_PER',
    'NUM_PER_FOR_PREM',
    'MULTI_DWELL_SIZE',
    #
    #'AGG_INCOME_GEO',
    #'NO_EARNINGS_GEO',
    #'BELOW_POVERTY_LVL_GEO',
    #'SNAP_GEO',
    #'ABOVE_GRD7_GEO',
    #'PUBLIC_ASSIST_GEO',
]

df = df[keep]
df.columns.to_list()

['MONTH',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'TOTAL_30_DAYS_AMT',
 'TOTAL_60_DAYS_AMT',
 'TOTAL_90_DAYS_AMT',
 'TOTAL_CUR_BALANCE',
 'SPA_PER_ID',
 'SPA_PREM_ID',
 'CMIS_MATCH',
 'HAS_COTENANT',
 'NUM_PREM_FOR_PER',
 'NUM_PER_FOR_PREM',
 'MULTI_DWELL_SIZE']

# Require Sequential Data Format
Problem  
* Currently a single person (SPA_PER_ID) sometimes has multiple attribute values for a given MONTH  

Cause  
* People have multiple premises (SPA_PREM_ID's) at the same time  

Solution  
* Create Person-Premis data objects to track

In [6]:
df.groupby(['SPA_PER_ID', 'SPA_PREM_ID', 'MONTH']).size().value_counts()

1    3231051
dtype: int64

In [7]:
df.groupby(['SPA_PER_ID', 'MONTH']).size().value_counts()

1    3216911
2       7070
dtype: int64

In [8]:
df['SPA_PER-PREM_ID'] = df.apply(lambda x: str(x['SPA_PER_ID']) + '-' + str(x['SPA_PREM_ID']), axis=1)

df = df.drop(['SPA_PER_ID', 'SPA_PREM_ID'], axis=1)

id_col = 'SPA_PER-PREM_ID'

df.groupby([id_col, 'MONTH']).size().value_counts()

1    3231051
dtype: int64

# Split X and Y, Training and Testing

In [9]:
TEST_FRAC = 1/3

# Split Train/Test sets
df_train, df_test = model_eval.split_on_people(df, id_col=id_col, test_frac=TEST_FRAC)

# Split X and Y
X_train = df_train.drop('CMIS_MATCH', axis=1)
Y_train = df_train[['CMIS_MATCH', id_col]].groupby(id_col).first()

X_test = df_test.drop('CMIS_MATCH', axis=1)
Y_test = df_test[['CMIS_MATCH', id_col]].groupby(id_col).first()

# Reformat to numpy ndarray

In [10]:
def reformat_for_ann(data: pd.DataFrame, id_col: str, time_col: str, max_sequence_length: int=0):
    '''
    09/02/21
    Input: Pandas DataFrame with and ID column, Time Column, and the rest variable columns
    Output: Numpy ndarray with dimensions: [num_data_objects, max_sequence_length, num_vars]
    Note: Padding is done at beginning of sequence.
    '''
    
    df = data.set_index([id_col, time_col]).copy()
    IDs = df.index.get_level_values(id_col).unique()
    VARs = df.columns
    NUM_VARS = len(VARs)
    MAX_SEQUENCE_LENGTH = None
    if max_sequence_length == 0:
        MAX_SEQUENCE_LENGTH = df.index.get_level_values(time_col).max() - df.index.get_level_values(time_col).min() + 1
    else:
        MAX_SEQUENCE_LENGTH = max_sequence_length
    NUM_IDs = df.index.get_level_values(id_col).nunique()

    data_array = np.zeros((NUM_IDs, MAX_SEQUENCE_LENGTH, NUM_VARS))

    print(f'NUM_IDs: {NUM_IDs}')
    print(f'MAX_SEQUENCE_LENGTH: {MAX_SEQUENCE_LENGTH}')
    print(f'NUM_VARS: {NUM_VARS}')
    print()
    
    for IDx in range(len(IDs)):
        person = df.loc[IDs[IDx]]
        num_person_timesteps = person.index.get_level_values(time_col).max() - person.index.get_level_values(time_col).min() + 1
        for VARx in range(len(VARs)):
            try:
                data_array[IDx, -num_person_timesteps:, VARx] = person[VARs[VARx]].values
            except:
                print(f'Attempted Destination\n{data_array[IDx, -num_person_timesteps:, VARx]}\n')
                print(f'Attempted Source\n{person[VARs[VARx]].values}')
                print(f'ID: {IDs[IDx]}')
                print(f'VAR: {VARs[VARx]}')
                print()
                return

    return data_array

In [11]:
#Y_train = Y_train.squeeze(axis=1)
#Y_test = Y_test.squeeze(axis=1)

print('X_train')
X_train = reformat_for_ann(X_train, id_col=id_col, time_col='MONTH')
print(f'Y_train\nIDs: {len(Y_train)}\n')

print('X_test')
X_test = reformat_for_ann(X_test, id_col=id_col, time_col='MONTH')
print(f'Y_test\nIDs: {len(Y_test)}')

X_train
NUM_IDs: 56811
MAX_SEQUENCE_LENGTH: 59
NUM_VARS: 17

Attempted Destination
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Attempted Source
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
ID: 125-75320
VAR: BREAK_ARRANGEMENT

Y_train
IDs: 56811

X_test
NUM_IDs: 28406
MAX_SEQUENCE_LENGTH: 59
NUM_VARS: 17

Attempted Destination
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Attempted Source
[0 0 0 0 0 0 0 0 0 0 0 0]
ID: 713-49137
VAR: BREAK_ARRANGEMENT

Y_test
IDs: 28406


In [12]:
X_train[:2]

TypeError: 'NoneType' object is not subscriptable

# Scale

In [ ]:
# Scaling 3D array
# New scaler for each variable

from sklearn.preprocessing import StandardScaler

scalers = {}
for i in range(X_train.shape[1]):
    scalers[i] = StandardScaler()
    X_train[:, i, :] = scalers[i].fit_transform(X_train[:, i, :]) 

for i in range(X_test.shape[1]):
    X_test[:, i, :] = scalers[i].transform(X_test[:, i, :]) 

# Time and Save

In [ ]:
data = {'X_train':X_train, 'Y_train':Y_train, 'X_test':X_test, 'Y_test':Y_test}

filename = 'ann_ready.pickle'
outfile = open(datapath+filename, 'wb')
pickle.dump(data, outfile)
outfile.close()

In [ ]:
from custom_methods.calc_time import calc_time_from_sec

calc_time_from_sec(time.time()-startTime)